# MVP workflow
- Data from clinicaltrials.gov

In [14]:
import json, os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spacy
import scispacy

ModuleNotFoundError: No module named 'spacy'

## Load json file with parsed information

In [2]:
# paths
path_to_json_file = os.path.abspath('../data/json/')

# json file
json_file = '/all_parsed_data_json' #name json file  
file = '{}{}.json'.format(path_to_json_file, json_file)

# load json file
df = pd.read_json(file)

In [3]:
df.head()

,nct_id,study_first_submitted,source,brief_title,overall_status,verification_date,study_type,study_first_posted,last_update_submitted,last_update_posted,...,detailed_description/textblock,brief_summary/textblock,location/facility/address/city,location/facility/address/country,location/facility/address/zip,sponsors/lead_sponsor/agency,sponsors/lead_sponsor/agency_class,study_design_info/allocation,study_design_info/intervention_model,study_design_info/primary_purpose
0,NCT01236404,"November 5, 2010",PhaseBio Pharmaceuticals Inc.,"Phase 1/2a, Randomized, Double-Blind, Placebo-...",Completed,May 2013,Interventional,"November 8, 2010","May 13, 2013","May 21, 2013",...,None,\n Primary objective:\n\n To evaluat...,Walnut Creek,United States,94598,PhaseBio Pharmaceuticals Inc.,Industry,Randomized,Parallel Assignment,Treatment
1,NCT00275600,"January 10, 2006",Mayo Clinic,Randomized Clinical Trial of Vitamin E and Eve...,Completed,November 2009,Interventional,"January 12, 2006","November 25, 2009","November 26, 2009",...,\n Cyclical mastalgia is a common complai...,\n This study is being done to find out w...,Rochester,United States,55905,Mayo Clinic,Other,Randomized,Parallel Assignment,Treatment
10,NCT01810211,"March 8, 2013",Nova Southeastern University,The Most Effective Intervention for Reducing P...,Completed,March 2014,Interventional,"March 13, 2013","September 24, 2014","September 26, 2014",...,None,"\n This study will look at which, if any,...",Raleigh,United States,27609,Nova Southeastern University,Other,Randomized,Parallel Assignment,Treatment
100,NCT00826111,"January 19, 2009",Steward St. Elizabeth's Medical Center of Bost...,The Effects of Eszopiclone and Lexapro on Pref...,Completed,June 2012,Interventional,"January 21, 2009","June 28, 2012","June 29, 2012",...,None,\n The study examined the effects of addi...,Boston,United States,02135,Steward St. Elizabeth's Medical Center of Bost...,Other,Randomized,Parallel Assignment,Treatment
1000,NCT02813850,"June 23, 2016",Assistance Publique - Hôpitaux de Paris,Oxygen Therapy and Pregnancy in Sickle Cell Di...,Recruiting,March 2017,Interventional,"June 27, 2016","December 20, 2017","December 21, 2017",...,\n Sickle cell disease (SCD) corresponds ...,\n The purpose of this study is to assess...,Paris,France,75015,Assistance Publique - Hôpitaux de Paris,Other,Randomized,Parallel Assignment,Prevention


In [5]:
df.shape

(5984, 24)

## Preprocess and clean data

In [6]:
# change column type
def data_types(dataframe, cols = [], to_type = ''):
    for col in cols:
        dataframe[col] = df[col].astype(to_type)

In [7]:
columns_dates = ['study_first_submitted', 'study_first_posted', 'last_update_submitted', 'last_update_posted', 'verification_date']
data_types(df, columns_dates, 'datetime64')

In [8]:
# Extract year 
df['year_submitted'] = df['study_first_submitted'].dt.year
df['year_last_updated'] = df['last_update_submitted'].dt.year

In [9]:
# Remove /n in all df
df = df.replace(r'\n',' ', regex=True)

In [10]:
# Create new column for all text
df['all_text'] = df['source'] + ' ' + df['brief_title'] + ' ' + df['condition'] + ' ' + df['condition_browse/mesh_term'] + ' '+ df['intervention_browse/mesh_term'] + ' '+ df['detailed_description/textblock'] + ' ' + df['brief_summary/textblock']

# All_text in lowercase
df['all_text'] = df['all_text'].str.lower()

# remove extra whitespace
df.all_text = df.all_text.replace('\s+', ' ', regex=True)


In [11]:
df['all_text'][0]

'phasebio pharmaceuticals inc. phase 1/2a, randomized, double-blind, placebo-controlled, study to assess safety, tolerability, pk and pd response of pb1023 injection following single and multiple sq doses in adults with type 2 diabetes mellitus diabetes mellitus, type 2 diabetes mellitus none none primary objective: to evaluate the safety and tolerability of single and multiple ascending doses of pb1023 administered as a subcutaneous (sc) injection in adult subjects with t2dm. secondary objectives: 1. to characterize the pharmacokinetic profile of pb1023 after single and multiple ascending doses of pb1023. 2. to assess the pharmacodynamic response of various single and multiple doses of pb1023 (daily fasting plasma glucose, and serial glucose, c-peptide and insulin levels in response to a liquid mixed meal tolerance test (mmtt). '

In [12]:
# Remove extra white space in summary
df['brief_summary/textblock'] = df['brief_summary/textblock'].replace('\s+', ' ', regex=True)
df['brief_summary/textblock'][0]


' Primary objective: To evaluate the safety and tolerability of single and multiple ascending doses of PB1023 administered as a subcutaneous (SC) injection in adult subjects with T2DM. Secondary objectives: 1. To characterize the pharmacokinetic profile of PB1023 after single and multiple ascending doses of PB1023. 2. To assess the pharmacodynamic response of various single and multiple doses of PB1023 (daily fasting plasma glucose, and serial glucose, c-peptide and insulin levels in response to a liquid Mixed Meal Tolerance Test (MMTT). '

## ML preprocessing

In [13]:
# Test: Extract entities with Scipacy
nlp = spacy.load('../data/en_core_sci_sm/en_core_sci_sm/')

doc = nlp(df.all_text[0])

NameError: name 'spacy' is not defined